In [32]:
# import libraries for data cleaning and visualization
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
from geopy.geocoders import Nominatim
import re

In [33]:
# epa_df contains dataset of epa violations
# cities contains dataset of all us cities from https://simplemaps.com/data/us-cities
epa_df = pd.read_csv('../datasets/epa_violations.csv',low_memory=False)
cities = pd.read_csv('../datasets/uscities.csv',low_memory=False)

In [34]:
epa_df.info()
cities.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40567 entries, 0 to 40566
Data columns (total 37 columns):
Company                                                      40567 non-null object
Parent Company                                               15087 non-null object
Penalty Amount                                               40567 non-null object
Subtraction From Penalty                                     40567 non-null object
Penalty Amount Adjusted For Eliminating Multiple Counting    40567 non-null object
Penalty Year                                                 40567 non-null int64
Penalty Date                                                 40567 non-null int64
Primary Offense                                              40567 non-null object
Secondary Offense                                            6 non-null object
Description                                                  24630 non-null object
Level of Government                                          40567 

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,population_proper,density,source,incorporated,timezone,zips,id
0,Prairie Ridge,Prairie Ridge,WA,Washington,53053,Pierce,47.1443,-122.1408,NaN,NaN,1349.8,polygon,False,America/Los_Angeles,98360 98391,1840037882
1,Edison,Edison,WA,Washington,53057,Skagit,48.5602,-122.4311,NaN,NaN,127.4,polygon,False,America/Los_Angeles,98232,1840017314
2,Packwood,Packwood,WA,Washington,53041,Lewis,46.6085,-121.6702,NaN,NaN,213.9,polygon,False,America/Los_Angeles,98361,1840025265
3,Wautauga Beach,Wautauga Beach,WA,Washington,53035,Kitsap,47.5862,-122.5482,NaN,NaN,261.7,point,False,America/Los_Angeles,98366,1840037725
4,Harper,Harper,WA,Washington,53035,Kitsap,47.5207,-122.5196,NaN,NaN,342.1,point,False,America/Los_Angeles,98366,1840037659


In [35]:
# Getting all rows where state or city value is empty
nan_fs = epa_df[epa_df['Facility State'].isnull() ].index
nan_add = epa_df[epa_df['City'].isnull() ].index

# Getting rid of the above rows so all the rows we deal with have a city and a state
epa_df.drop(nan_add, inplace=True)
# epa_df.drop(nan_fs, inplace=True)

# Create a second data frame with relevant information
df2 = epa_df[['Company', 'Penalty Amount', 'Facility State', 'City']]

nan_industry = epa_df[epa_df['Specific Industry of Parent'].isnull() ].index
epa_df.drop(nan_industry, inplace=True)
# Create third df with parent industry
df3 = epa_df[['Company', 'Penalty Amount', 'Facility State', 'City', 'Specific Industry of Parent']]

# df2.dtypes
# Make a deep copy of above data frame
df2 = df2.copy(deep=True)
df3 = df3.copy(deep=True)

# All the values under penalty amount column are list, and also have '$' appended to the front of the value,
# We want to convert this entire column to ints because this will be necessary when we display our map
penalty = df2['Penalty Amount'].tolist()
float_penalties = []
for amount in penalty:
    cleaned_amt = re.sub("[^0-9|.]", "", amount)
    float_penalties.append(cleaned_amt)
df2['Penalty Amount'] = float_penalties
df2.head()

penalty = df3['Penalty Amount'].tolist()
float_penalties = []
for amount in penalty:
    cleaned_amt = re.sub("[^0-9|.]", "", amount)
    float_penalties.append(cleaned_amt)
df3['Penalty Amount'] = float_penalties
df3.head()

,Company,Penalty Amount,Facility State,City,Specific Industry of Parent
8,Action Manufacturing Company,1200000,Pennsylvania,Atglen,aerospace & military contracting
11,Advanced Sterilization Products,136800,California,Irvine,pharmaceuticals
15,Ag Processing Inc,96588,Iowa,Mason City,agribusiness
17,Agrifos Fertilizer LLC,1800000,Texas,Pasadena,biofuels; fertilizers
18,Agrifos Fertilizer LLC,385000,Texas,Pasadena,biofuels; fertilizers


In [36]:
# This will be a cleaned data frame that we will use for the actual visualization
# Lat and lng coordinates needed to plot locations
cleaned_df = pd.DataFrame(columns=['company','penalty','lat','lon'])
ind_cleaned_df = pd.DataFrame(columns=['company','penalty','lat','lon'])

cleaned_df.head()

,company,penalty,lat,lon


In [252]:
# an example dict to append to our new df
# This row is not added to df, this is just to make sure that I can add a new row to my data frame
new = {'company' : 'Amazon' , 'penalty' : 22222, 'lat': 1234.2, 'lon': 234.14}
cleaned_df.append(new, ignore_index=True)

,company,penalty,lat,lon
0,Amazon,22222,1234.2,234.14


In [ ]:
# Iterating through dataframe and checking for lat & lng coords with cities database
# If we have a match, find coords and add relevant info to dataframe
# If we don't have a match, that might mean we have some bad data, we want to ignore it
locations = []
for row in df2.itertuples():
    if isinstance(city_name, str) and isinstance(row._3, str):
        city_name = str(row.City).lower().title()
        city_row = cities.loc[(cities['city'] == city_name) & (cities['state_name'] == row._3)]

        if city_row.empty:
            a = 2
        else:
#             print('match found')
            lat = (city_row['lat'].values)[0]
            lng = (city_row['lng'].values)[0]
            company = (row.Company)
            penalty = (row._2)
            new_row = {'company': company, 
                       'penalty': penalty, 
                       'lat': lat, 
                       'lon': lng,
                       'city': city_name,
                       'state': row._3}
            cleaned_df = cleaned_df.append(new_row, ignore_index=True)
        
cleaned_df.head()

In [37]:
locations = []
for row in df3.itertuples():
    city_name = str(row.City).lower().title()
    if isinstance(city_name, str) and isinstance(row._3, str):
        city_row = cities.loc[(cities['city'] == city_name) & (cities['state_name'] == row._3)]

        if city_row.empty:
            a = 2
        else:
#             print('match found')
            lat = (city_row['lat'].values)[0]
            lng = (city_row['lng'].values)[0]
            company = (row.Company)
            penalty = (row._2)
            new_row = {'company': company, 
                       'penalty': penalty, 
                       'lat': lat, 
                       'lon': lng,
                       'city': city_name,
                       'state': row._3,
                       'industry': row._5}
            ind_cleaned_df = ind_cleaned_df.append(new_row, ignore_index=True)
        
ind_cleaned_df.head()

,company,penalty,lat,lon,city,industry,state
0,Action Manufacturing Company,1200000,39.9473,-75.9754,Atglen,aerospace & military contracting,Pennsylvania
1,Advanced Sterilization Products,136800,33.6772,-117.7738,Irvine,pharmaceuticals,California
2,Ag Processing Inc,96588,43.1487,-93.1998,Mason City,agribusiness,Iowa
3,Agrifos Fertilizer LLC,1800000,29.6584,-95.1499,Pasadena,biofuels; fertilizers,Texas
4,Agrifos Fertilizer LLC,385000,29.6584,-95.1499,Pasadena,biofuels; fertilizers,Texas


In [38]:
# The block of code that gets all the lat/lng coordinates takes a long time to run, so we will save the result in a csv file
cleaned_df.to_csv('epa_map.csv', index=False)
ind_cleaned_df.to_csv('epa_ind_map.csv',index=False)

In [39]:
# Read in csv file, and we want to sort values so that the violation with the highest penalty is at the top
epa_map = pd.read_csv('epa_map.csv',low_memory=False)
epa_map = epa_map.sort_values(by='penalty',ascending=False)
epa_map.head()

epa_ind_map = pd.read_csv('epa_ind_map.csv',low_memory=False)
epa_ind_map = epa_ind_map.sort_values(by='penalty',ascending=False)
epa_ind_map.head(n=10)

,company,penalty,lat,lon,city,industry,state
8712,VOLKSWAGEN OF AMERICA,1450000000,42.6735,-83.2447,Auburn Hills,motor vehicles,Michigan
22,BP North America Inc.,408000000,41.6731,-87.4843,Whiting,oil & gas,Indiana
9,Alcoa Inc.,334000000,30.6543,-97.0088,Rockdale,metals,Texas
1922,BP Products North America Inc.,179000000,29.4128,-94.9658,Texas City,oil & gas,Texas
38,CF Industries Inc.,176201500,28.0144,-82.1200,Plant City,chemicals,Florida
8696,Shell Oil,117600000,29.6898,-95.1151,Deer Park,oil & gas,Texas
7392,HYUNDAI MOTOR AMERICA,100000000,33.7105,-117.9514,Fountain Valley,motor vehicles,California
3140,Citgo Petroleum Corporation,90000000,30.2030,-93.2150,Lake Charles,oil & gas,Louisiana
8537,VOLVO TRUCK CORPORATION,76625640,36.0956,-79.8269,Greensboro,motor vehicles,North Carolina
4202,Honeywell Resins and Chemicals LLC,69000000,37.2914,-77.2985,Hopewell,diversified,Virginia


In [264]:
# Setting the text of each marker so that when we hover over a bubble, we know the company, penalty, and location
# The skeleton of how to produce a bubble map can be found on plotly, https://plot.ly/python/bubble-maps/
# We are going to separate groups into the following order: top 50, 51-1000, etc
# Epa penalties = very big so we want to scale these values on the map. Otherwise we won't get a map and instead we'll get very large circles.
# then for every category, we want to create a marker
# once we have all relevant info for an epa violation, we will append it to a list

print(epa_map.shape[0])
epa_map['text'] = epa_map['company'] + '<br>' + epa_map['city'] + ', ' + epa_map['state'] + '<br>Penalty ' + (epa_map['penalty']/1e6).astype(str)+' million'
limits = [(0,50),(51,1000),(1001,10000),(10001,20000),(20001,26583)]
colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
cities = []
scale = 100000

for i in range(len(limits)):
    lim = limits[i]
    df_sub = epa_map[lim[0]:lim[1]]
#     print(df_sub.head())
    city = go.Scattergeo(
        locationmode = 'USA-states',
        lon = df_sub['lon'],
        lat = df_sub['lat'],
        text = df_sub['text'],
        marker = go.scattergeo.Marker(
            size = df_sub['penalty']/scale,
            color = colors[i],
            line = go.scattergeo.marker.Line(
                width=0.5, color='rgb(40,40,40)'
            ),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]) )
    cities.append(city)
print(cities)

26583
[Scattergeo({
    'lat': array([42.6735, 41.6731, 30.6543, 29.4128, 28.0144, 29.6898, 33.7105, 34.037 ,
                  30.203 , 36.0956, 37.2914, 38.734 , 29.4128, 40.5669, 63.9454, 29.8554,
                  29.7868, 35.847 , 42.4438, 39.2091, 37.7562, 39.5743, 30.424 , 39.6401,
                  33.8346, 27.7173, 35.1047, 40.8439, 37.9364, 41.8743, 41.8743, 40.8731,
                  34.1945, 29.4128, 43.851 , 42.4184, 36.0145, 39.6401, 43.5903, 43.064 ,
                  39.6067, 37.052 , 38.5319, 42.7229, 29.8554, 30.0687, 29.6646, 30.229 ,
                  33.9447, 33.5646]),
    'locationmode': 'USA-states',
    'lon': array([ -83.2447,  -87.4843,  -97.0088,  -94.9658,  -82.12  ,  -95.1151,
                  -117.9514,  -80.9042,  -93.215 ,  -79.8269,  -77.2985,  -85.0612,
                   -94.9658,  -74.2475, -149.0152,  -93.9264,  -95.3905,  -77.0659,
                   -91.0091,  -85.918 , -122.443 ,  -75.5939,  -88.5289,  -77.7217,
                  -118.3417,  -9

In [43]:
print(epa_ind_map.shape[0])
epa_ind_map['text'] = epa_ind_map['company'] + '<br>' + epa_ind_map['city'] + ', ' + epa_ind_map['state'] + '<br>Penalty ' + (epa_ind_map['penalty']/1e6).astype(str)+' million' + '<br>Industry ' + epa_ind_map['industry']
ind_limits = [(0,250),(251,500),(501,1000),(1001,5000),(5001,9069)]
ind_colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
ind_cities = []
ind_scale = 100000

for i in range(len(ind_limits)):
    lim = limits[i]
    df_sub = epa_ind_map[lim[0]:lim[1]]
#     print(df_sub.head())
    city = go.Scattergeo(
        locationmode = 'USA-states',
        lon = df_sub['lon'],
        lat = df_sub['lat'],
        text = df_sub['text'],
        marker = go.scattergeo.Marker(
            size = df_sub['penalty']/ind_scale,
            color = ind_colors[i],
            line = go.scattergeo.marker.Line(
                width=0.5, color='rgb(40,40,40)'
            ),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]) )
    ind_cities.append(city)
print(ind_cities)

9069
[Scattergeo({
    'lat': array([42.6735, 41.6731, 30.6543, 29.4128, 28.0144, 29.6898, 33.7105, 30.203 ,
                  36.0956, 37.2914, 38.734 , 29.4128, 40.5669, 63.9454, 29.8554, 29.7868,
                  39.2091, 37.7562, 39.5743, 30.424 , 39.6401, 33.8346, 27.7173, 35.1047,
                  40.8439, 37.9364, 41.8743, 41.8743, 40.8731, 34.1945, 29.4128, 43.851 ,
                  42.4184, 36.0145, 39.6401, 43.5903, 43.064 , 37.052 , 38.5319, 42.7229,
                  29.8554, 30.229 , 29.6646, 33.9447, 33.5646, 33.5276, 41.9667, 33.9169,
                  33.9169, 42.0695]),
    'locationmode': 'USA-states',
    'lon': array([ -83.2447,  -87.4843,  -97.0088,  -94.9658,  -82.12  ,  -95.1151,
                  -117.9514,  -93.215 ,  -79.8269,  -77.2985,  -85.0612,  -94.9658,
                   -74.2475, -149.0152,  -93.9264,  -95.3905,  -85.918 , -122.443 ,
                   -75.5939,  -88.5289,  -77.7217, -118.3417,  -97.3822,  -89.9773,
                  -111.9188,  -87

In [44]:
# create the layout for the visualization
layout = go.Layout(
        title = go.layout.Title(
            text = 'EPA Violations 2000-2019<br>(Click legend to toggle traces)'
        ),
        showlegend = True,
        geo = go.layout.Geo(
            scope = 'usa',
            projection = go.layout.geo.Projection(
                type='albers usa'
            ),
            showland = True,
            subunitwidth=1,
            countrywidth=1,
        )
    )

ind_layout = go.Layout(
        title = go.layout.Title(
            text = 'EPA Violations 2000-2019 (Industries)<br>(Click legend to toggle traces)'
        ),
        showlegend = True,
        geo = go.layout.Geo(
            scope = 'usa',
            projection = go.layout.geo.Projection(
                type='albers usa'
            ),
            showland = True,
            subunitwidth=1,
            countrywidth=1,
        )
    )

In [268]:
# plot visualization
fig = go.Figure(data=cities, layout=layout)
py.iplot(fig, filename='d3-bubble-map-populations')

In [45]:
# plot visualization
fig = go.Figure(data=ind_cities, layout=ind_layout)
py.iplot(fig, filename='d3-bubble-map-populations')